# Amazon Fashion Category - Data Exploration

Category: Amazon Fashion
Team Member: Viktoriia Babii

In [2]:
import os
from pathlib import Path
from datetime import datetime, timedelta
import warnings

import rootutils
rootutils.setup_root(Path.cwd(), indicator=".project-root", pythonpath=True)

ROOT_DIR = Path(os.environ.get("PROJECT_ROOT", Path.cwd()))
CLEANED_REVIEWS_PATH = ROOT_DIR / "data/cleaned/review_categories/fashion_reviews_cleaned.parquet"
CLEANED_METADATA_PATH = ROOT_DIR / "data/cleaned/meta_categories/fashion_metadata_cleaned.parquet"

warnings.filterwarnings("ignore")

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from src.amazon_reviews_analysis.utils import build_spark

# Initialize Spark
spark = build_spark()
print("✓ Spark Session created successfully!")

✓ Spark Session created successfully!


In [4]:
print("📂 Loading cleaned datasets...")
reviews_df = spark.read.parquet(str(CLEANED_REVIEWS_PATH))
metadata_df = spark.read.parquet(str(CLEANED_METADATA_PATH))

print(f"Reviews: {reviews_df.count():,} records")
print(f"Metadata: {metadata_df.count():,} records")

📂 Loading cleaned datasets...
Reviews: 2,473,301 records
Metadata: 826,050 records


## Q1 "Misleading" High Rating

In [5]:
from pyspark.sql.functions import col, avg, count, when, lower, format_number, desc

print("\n--- Products with 'Misleading' High Ratings ---")

# --- 1. Keyword Logic ---
negative_keywords = ["broken", "fake", "poor", "wrong"]

keyword_condition = None
for kw in negative_keywords:
    condition = lower(col("text")).like(f"%{kw}%") 
    if keyword_condition is None:
        keyword_condition = condition
    else:
        keyword_condition = keyword_condition | condition

# This case will flag reviews that are 1-star AND contain a negative keyword
problematic_1_star_case = when((col("rating") == 1) & keyword_condition, 1)


# --- 2. Join Preparation ---
reviews_compact = reviews_df.select(
    "parent_asin", 
    "rating", 
    "text"
)

metadata_compact = metadata_df.select(
    "parent_asin", 
    "title", # Product Title
    "price"  # Needed for the filter
)

# --- 3. Join and Execution ---

joined_df = reviews_compact.join(
    metadata_compact,
    "parent_asin",
    "inner"
)

q2_result = (
    joined_df
    .where(col("price") > 75)
    
    .groupBy("parent_asin", "title")
    
    .agg(
        avg("rating").alias("avg_rating"),
        count(when(col("rating") == 1, 1)).alias("total_1_star_reviews"),
        count(problematic_1_star_case).alias("negative_keyword_1_star")
    )
    
    .where(
        (col("avg_rating") >= 4.0) &
        (col("total_1_star_reviews") > 0) & 
        ((col("negative_keyword_1_star") / col("total_1_star_reviews")) > 0.10)
    )
    
    # Add a percentage column for clarity
    .withColumn("problematic_ratio", col("negative_keyword_1_star") / col("total_1_star_reviews"))
    
    # Sort by the count of problematic reviews and take top 20
    .orderBy(desc("negative_keyword_1_star"))
    .limit(20)
    
    # Select and format the final columns for readability
    .select(
        "title",
        format_number("avg_rating", 2).alias("avg_rating"),
        "total_1_star_reviews",
        "negative_keyword_1_star",
        format_number("problematic_ratio", 3).alias("problematic_ratio")
    )
)

# 4. Show the result
q2_result.show(truncate=False)


--- Products with 'Misleading' High Ratings ---
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------+-----------------------+-----------------+
|title                                                                                                                                                                                                 |avg_rating|total_1_star_reviews|negative_keyword_1_star|problematic_ratio|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------+-----------------------+-----------------+
|wikiwiki S Series Cantilever Patio Umbrellas Outdoor Offset Umbrella w/Fade & UV Resistant Solution-dyed Fabric, 5 Level 360 

This table identifies a "hidden problem" product list. These are expensive items (over $75) that appear to be high-quality, all having an average rating above 4.0 stars. However, this high rating is often misleading. We discovered these by analyzing their 1-star reviews for specific, serious complaints like "broken," "fake," "poor," or "wrong."

The key metric is the problematic_ratio, which shows the percentage of 1-star reviews that contain these specific complaints. When this ratio is 1.000, as seen with the Ray-Ban Sunglasses or the Allmygold... Rose, it's a major red flag. It means 100% of the item's 1-star reviews contained a severe complaint, strongly suggesting customers are receiving "fake" or "broken" products despite the high 4.47-star average rating.

This isn't just about a 1.000 ratio, either. The top item, the wikiwiki... Patio Umbrella, has a great 4.61 rating, but 25% of its 20 1-star reviews were "problematic." This indicates a systemic quality trend and a high risk for buyers, not just a few random bad experiences. Overall, this list is a high-priority for a Quality Assurance (QA) team, as it pinpoints products where a high rating is effectively masking significant defects or potential counterfeit issues that require immediate manual investigation.

## Q2 "Most Loyal" Users (Top 5%)

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, datediff, avg, count, percent_rank, format_number, desc, to_timestamp

print("\n--- Top 5% 'Most Loyal' Users (FIXED) ---")

# Step 1: Window Function (1) + Logic (DATEDIFF)

reviews_with_ts = reviews_df.withColumn(
    "review_timestamp", 
    to_timestamp(col("timestamp") / 1000) 
)

user_window = Window.partitionBy("user_id").orderBy(col("review_timestamp"))

reviews_with_gaps = (
    reviews_with_ts.select("user_id", "review_timestamp")
    .withColumn("prev_review_time", lag("review_timestamp").over(user_window))
    .withColumn("days_diff", datediff(col("review_timestamp"), col("prev_review_time")))
)

# Step 2: Group By + Filter (HAVING) + Logic (AVG)

user_loyalty_stats = (
    reviews_with_gaps
    .groupBy("user_id")
    .agg(
        avg("days_diff").alias("avg_days_between_reviews"),
        count("*").alias("total_reviews") 
    )
    .where(col("total_reviews") >= 15)
)

# Step 3: Window Function (2) + Filter

ranking_window = Window.orderBy(col("avg_days_between_reviews").asc())

top_5_percent_loyal = (
    user_loyalty_stats
    .withColumn("percent_rank", percent_rank().over(ranking_window))
    .where(col("percent_rank") <= 0.05)
    .orderBy(col("percent_rank"))
)

# Step 4: Show Results

print(f"Found {top_5_percent_loyal.count()} users in the top 5% (out of {user_loyalty_stats.count()} users with 15+ reviews).")

q1_result = top_5_percent_loyal.select(
    "user_id",
    format_number("avg_days_between_reviews", 2).alias("avg_days_between_reviews"),
    "total_reviews",
    format_number("percent_rank", 4).alias("percent_rank")
)

q1_result.show(50, truncate=False)


--- Top 5% 'Most Loyal' Users (FIXED) ---
Found 38 users in the top 5% (out of 744 users with 15+ reviews).
+------------------------------+------------------------+-------------+------------+
|user_id                       |avg_days_between_reviews|total_reviews|percent_rank|
+------------------------------+------------------------+-------------+------------+
|AEM5PO3PITEPDNNATSGM2PD6C62Q  |0.00                    |18           |0.0000      |
|AEKEF7PBXCP345X2UPV3IJY6QMNQ  |1.38                    |22           |0.0013      |
|AFQNKOJVCTSPUXQIHHKIZHUC4U2Q  |3.06                    |17           |0.0027      |
|AFQQQ5LGNSQUEBGDCYBAZZE5T3DA  |4.32                    |145          |0.0040      |
|AGZZXSMMS4WRHHJRBUJZI4FZDHKQ  |5.18                    |311          |0.0054      |
|AGMZTGMF4RCSZL55QIN6RKQZ4BKA  |7.00                    |18           |0.0067      |
|AFW373BQ7S5JTCI37O2TFDKAAEDQ  |7.15                    |21           |0.0081      |
|AFR2XB7TRMQJCRB6INYP2W73EH5A  |7.24     

This query was designed to find the top 5% most "loyal" users in the Amazon Fashion dataset, defining loyalty as the shortest average time between reviews. To do this, we first calculated the day gap between each user's consecutive reviews. We then grouped these gaps by user to find their average time and total review count. After filtering to keep only the 744 active users with 15 or more reviews, we ranked them by this average gap (lowest is best) and selected the top 5%, which resulted in 38 users.

The query highlighted one clear anomaly: the top user has an average review time of 0.00 days. This is not "loyalty" but likely spam or "bulk uploading," as all 18 reviews were probably posted on the exact same day.

However, the rest of this top 5% group are true "super users." For example, the user we previously discussed, who has 311 review, has a realistic average gap of just 5.18 days. Other users in this group with 145 reviews, write feedback every 4.32 days.

Therefore, the analysis successfully proved that while most customers are not frequent reviewers, there is a clearly identifiable and highly engaged group of "brand ambassadors" who provide regular, active feedback.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    col, count, sum, avg, desc, 
    year, month, to_timestamp, format_number
)

print("\n--- [Query 3] Helpfulness Over Time for Top 5 Stores ---")

# Step 1: Find the top 5 most-reviewed stores
print("Finding top 5 stores...")

reviews_select = reviews_df.select("parent_asin")
meta_select = metadata_df.select("parent_asin", "store")

top_5_stores_df = (
    reviews_select
    .join(meta_select, "parent_asin")
    .where(col("store").isNotNull() & (col("store") != ""))
    .groupBy("store")
    .count()
    .orderBy(desc("count"))
    .limit(5)
)

# Collect the list of names for filtering
top_5_store_list = [row.store for row in top_5_stores_df.collect()]
print(f"Top 5 stores found: {top_5_store_list}")

# Step 2: Prepare and filter data for these 5 stores

reviews_with_ts = reviews_df.withColumn(
    "review_date", 
    to_timestamp(col("timestamp") / 1000)
).select("parent_asin", "review_date", "helpful_vote")

# Join reviews with store info, filtering for only the top 5
filtered_reviews = (
    reviews_with_ts
    .join(meta_select, "parent_asin")
    .where(col("store").isin(top_5_store_list)) # Filter for top 5
)

# Step 3: Group by store and month, calculating monthly aggregates
monthly_agg = (
    filtered_reviews
    .withColumn("year", year(col("review_date")))
    .withColumn("month", month(col("review_date")))
    .groupBy("store", "year", "month")
    .agg(
        sum("helpful_vote").alias("monthly_helpful_votes"),
        count("*").alias("monthly_reviews")
    )
)

# Step 4: Apply window function for cumulative sum & calculate monthly avg
store_time_window = (
    Window.partitionBy("store")
    .orderBy("year", "month")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

final_stats = (
    monthly_agg
    .withColumn(
        "avg_helpful_per_review_monthly", 
        col("monthly_helpful_votes") / col("monthly_reviews")
    )
    .withColumn(
        "cumulative_helpful_votes", 
        sum("monthly_helpful_votes").over(store_time_window)
    )
    .orderBy("store", "year", "month")
)

# Step 5: Show the results
q3_result = final_stats.select(
    "store", "year", "month",
    format_number("avg_helpful_per_review_monthly", 2).alias("avg_helpful_this_month"),
    col("cumulative_helpful_votes").alias("running_total_helpful_votes")
)

q3_result.show(100, truncate=False)


--- [Query 3] Helpfulness Over Time for Top 5 Stores ---
Finding top 5 stores...
Top 5 stores found: ['NELEUS', 'Romwe', 'Casio', 'Nike', 'CHUVORA']
+-------+----+-----+----------------------+---------------------------+
|store  |year|month|avg_helpful_this_month|running_total_helpful_votes|
+-------+----+-----+----------------------+---------------------------+
|CHUVORA|2010|12   |0.00                  |0                          |
|CHUVORA|2011|1    |7.50                  |15                         |
|CHUVORA|2011|5    |2.25                  |24                         |
|CHUVORA|2011|6    |1.00                  |26                         |
|CHUVORA|2011|7    |3.00                  |29                         |
|CHUVORA|2011|8    |1.50                  |32                         |
|CHUVORA|2011|9    |0.00                  |32                         |
|CHUVORA|2011|10   |2.00                  |34                         |
|CHUVORA|2011|11   |3.50                  |41             

This query tracks the "helpfulness" of reviews over time for the top 5 most-reviewed stores. First, it finds the 5 stores with the most reviews by joining reviews_df and metadata_df, grouping by store, counting reviews, and taking the top 5. It filters the entire review dataset to only include reviews for those top 5 stores. It groups these filtered reviews by store, year, and month. For each group, it calculates the sum() of helpful_vote and the count() of reviews. Finally, it uses a Window function to calculate two metrics for each month.

avg_helpful_this_month: The average helpful votes per review, showing the current trend.
running_total_helpful_votes: A cumulative sum of all helpful votes, showing the store's historical legacy.

Results Analysis
The results show a clear and dramatic trend: the usefulness of new reviews drops to almost zero over time. This is a classic case of "review saturation."

Golden Era (2011-2013): When the store/products were new, reviews were highly valuable. The avg_helpful_this_month was high, often between 3.00 and 13.00.

Saturation Era (2014-2017): As hundreds of reviews accumulated, new ones got "lost in the crowd." The average helpfulness dropped to below 1.0.

Ignored Era (2018-2019): With thousands of reviews, new ones are effectively invisible. The average helpfulness plummeted to ~0.1 to 0.2.

For a mature store like CHUVORA, new reviews have almost no impact. Customers are no longer reading them; they are only reading the "Top Reviews," which were likely written back in the "Golden Era."

In [9]:
from pyspark.sql.functions import col, avg, length, format_number

print("\n--- Average Review Length by Star Rating ---")

q4_result = (
    reviews_df
    # 1. Create a new column 'text_length' by calculating the length of the 'text'
    .withColumn("text_length", length(col("text")))
    
    # 2. Group By: Group the data by the 'rating'
    .groupBy("rating")
    
    # 3. Logic (Agg): Calculate the average of the 'text_length'
    .agg(
        avg("text_length").alias("avg_review_length")
    )
    
    # Order by rating (1 to 5) for a clear view
    .orderBy("rating")
    
    # Format the number to 2 decimal places
    .select(
        "rating",
        format_number("avg_review_length", 2).alias("avg_review_length_chars")
    )
)

# Show the results
q4_result.show()


--- Average Review Length by Star Rating ---
+------+-----------------------+
|rating|avg_review_length_chars|
+------+-----------------------+
|   1.0|                 141.25|
|   2.0|                 169.03|
|   3.0|                 172.64|
|   4.0|                 177.56|
|   5.0|                 125.55|
+------+-----------------------+



The analysis reveals that the longest and most detailed feedback is not found in 1-star or 5-star reviews, but in 3 and 4-star reviews.

Happy customers (5-star) are the most concise (125 chars), typically writing short phrases like "Love it!" Similarly, very unhappy customers (1-star) are also brief (141 chars), focusing on the problem like "Broken!"

The most valuable, constructive criticism comes from customers with mixed feelings (3 and 4-stars), who write significantly longer reviews (172-177 chars) to explain what they liked and disliked. This tells a business to focus on 3 and 4-star reviews to find the most actionable insights for product improvement.

In [10]:
from pyspark.sql.functions import col, count, desc

print("\n--- Products with 'Silent' 5-Star Reviews (0 helpful votes) ---")

# 1. "Smart Select" to prepare for the join and avoid ambiguous columns
# We only need rating and helpful_vote from reviews
reviews_select = reviews_df.select("parent_asin", "rating", "helpful_vote")

# We only need title from metadata
# (We already handled the 'title' vs 'review_title' ambiguity in a previous step)
meta_select = metadata_df.select("parent_asin", "title")

# 2. Join: Join the two DataFrames
joined_df = reviews_select.join(meta_select, "parent_asin")

# 3. Filter (2x): 
#    - WHERE rating = 5
#    - AND helpful_vote = 0
filtered_df = joined_df.where(
    (col("rating") == 5) & (col("helpful_vote") == 0)
)

# 4. GroupBy & Agg: Group by the product title and count how many such reviews each product has
q5_result = (
    filtered_df
    .groupBy("title")
    .agg(
        count("*").alias("silent_5star_review_count")
    )
    .orderBy(desc("silent_5star_review_count")) # Show the worst offenders first
)

# Show the top 20 products
q5_result.show(20, truncate=False)


--- Products with 'Silent' 5-Star Reviews (0 helpful votes) ---
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+
|title                                                                                                                                                                                                   |silent_5star_review_count|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+
|Casio F91W-1 Classic Resin Strap Digital Sport Watch                                                                                                                                                    |4783                     |
|Russell Athletic M

This table identifies products with the highest volume of 5-star reviews that no one found helpful (0 helpful_vote). This is a 'red flag' that reveals two key problems:

Low-Quality Feedback: Many 5-star reviews are likely "empty" (e.g., "Great!"), which don't help new customers.

Review Saturation: For popular items like the Casio watch, new 5-star reviews are posted but never seen or upvoted, getting "lost" at the bottom of a long list.

Business Conclusion: A high count of these 'silent' 5-star reviews suggests that the feedback is not valuable and can be a strong indicator of fake or "bot" activity used to inflate ratings.

In [14]:
from pyspark.sql.functions import col, year, to_timestamp, lower, count, when, format_number

print("\n--- [Query 6] Sizing-Related vs. Total Reviews (2022-2024) ---")

# 1. Convert timestamp (bigint ms) to a proper date
reviews_with_ts = reviews_df.withColumn(
    "review_date", 
    to_timestamp(col("timestamp") / 1000)
)

# 2. Filter for the years 2022, 2023, and 2024
reviews_filtered_df = reviews_with_ts.where(
    year(col("review_date")).isin([2021, 2022, 2023])
)

# 3. Group by year and calculate both counts in one pass
q6_result_df = (
    reviews_filtered_df
    .groupBy(year(col("review_date")).alias("year"))
    .agg(
        # Calculation 1: Get the TOTAL count for the year
        count("*").alias("total_reviews"),
        
        # Calculation 2: Get the count for 'size' reviews
        count(
            when(lower(col("text")).like("%size%"), True)
        ).alias("size_reviews")
    )
    # 4. Calculate percentage
    .withColumn(
        "percentage_size",
        (col("size_reviews") / col("total_reviews")) * 100
    )
    .orderBy("year")
)

# 5. Show the results
print("Sizing-related review statistics for 2022, 2023, and 2024:")
q6_result_df.select(
    "year",
    "total_reviews",
    "size_reviews",
    format_number("percentage_size", 2).alias("percentage_size")
).show()


--- [Query 6] Sizing-Related vs. Total Reviews (2022-2024) ---
Sizing-related review statistics for 2022, 2023, and 2024:
+----+-------------+------------+---------------+
|year|total_reviews|size_reviews|percentage_size|
+----+-------------+------------+---------------+
|2021|       305983|       39854|          13.02|
|2022|       171499|       23177|          13.51|
|2023|        35430|        4296|          12.13|
+----+-------------+------------+---------------+

